In [1]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
# from imgaug import augmenters as iaa
from scipy import ndimage
from math import ceil, floor, log
import cv2
import tensorflow as tf
import tensorflow.keras as keras
# import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
import heapq
import efficientnet.tfkeras as efn 
# from efficientnet.tfkeras import EfficientNetB0
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

from sklearn.model_selection import ShuffleSplit


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
tf.test.is_gpu_available()

True

In [3]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def preprocess (img):
    img /= 255
    centered = np.subtract(img, imagenet_mean)
    standardized = np.divide(centered, imagenet_std)
    return standardized 
    return img   

In [4]:
train_datagen = ImageDataGenerator(
                                   preprocessing_function=preprocess,
#                                    height_shift_range=0.05,
#                                    width_shift_range=0.05,
                                   rotation_range=20,
                                   zoom_range=0.05,
                                   brightness_range=[0.9,1.1],
                                   fill_mode='constant',
#                                    horizontal_flip=True
                                                                            )
test_datagen = ImageDataGenerator(
                                  preprocessing_function=preprocess
                                                                            )

In [5]:
train_generator = train_datagen.flow_from_directory('C:/Users/physics/Documents/NMHFiles_224/Train', 
                                                    target_size=(224, 224), 
                                                    class_mode= 'binary', 
                                                    color_mode='rgb', 
                                                    batch_size=20,
                                                    interpolation ='lanczos'
                                                    )
validation_generator = test_datagen.flow_from_directory('C:/Users/physics/Documents/NMHFiles_224/Validate', 
                                                        target_size=(224, 224), 
                                                        class_mode= 'binary',
                                                        color_mode='rgb', 
                                                        batch_size=20,
                                                        interpolation ='lanczos'
                                                        )

Found 5656 images belonging to 2 classes.
Found 1887 images belonging to 2 classes.


In [6]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(2, 5, figsize=(40,40))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [7]:
# sample_training_images, _ = next(train_generator)
# sample_training_images = np.multiply(sample_training_images, imagenet_std)
# sample_training_images = np.add(sample_training_images, imagenet_mean)
# plotImages(sample_training_images)

In [2]:
base_model = efn.EfficientNetB2(include_top=False, weights= None, input_shape=(224,224,3),
                             backend = keras.backend, layers = keras.layers, models = keras.models, utils = keras.utils)
x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(base_model.output)
x = keras.layers.Dropout(0.125)(x)
predictions = keras.layers.Dense(1, activation="sigmoid", name='last')(x)
model = keras.models.Model(inputs=base_model.input, outputs=predictions)
model.summary()     

In [9]:
model.load_weights('')

In [10]:
len(base_model.layers)

332

In [11]:
for layer in model.layers[:332]:
    layer.trainable = False
for layer in model.layers[332:]:
    layer.trainable = True

In [12]:
model.compile(loss='binary_crossentropy', 
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9,nesterov=True),
              metrics=['acc',                       
                     keras.metrics.AUC(),
                     keras.metrics.Precision(name='precision'),
                     keras.metrics.Recall(name='recall')])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
weightpath = 'auc_weight_NIH_224.hdf5'
es = EarlyStopping(monitor='val_auc', 
                   verbose=1, 
                   patience=10, 
                   min_delta=0.001, 
                   mode='max')
mc = ModelCheckpoint(weightpath, 
                     monitor='val_auc', 
                     verbose=1, 
                     save_best_only=True, 
                     mode='max')
rlr = ReduceLROnPlateau(monitor='val_auc',
                        mode='max',
                        factor=0.1,
                        patience=3)

In [14]:
class_weights = {0: (5656/4030)/2,
                1: (5656/1626)/2}
print(class_weights)



{0: 0.7017369727047147, 1: 1.7392373923739237}


In [15]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [3]:
history = model.fit_generator(train_generator, 
                              epochs = 30, 
                              validation_data=validation_generator, 
                              class_weight = class_weights, 
                             callbacks = [es, mc,rlr])

In [4]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [5]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [6]:
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model auc')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [7]:
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [8]:
plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.title('Model recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [23]:
model.load_weights('auc_weight_NIH_224.hdf5')

In [24]:
for layer in model.layers[:]:
    layer.trainable = True

In [25]:
es = EarlyStopping(monitor='val_auc', 
                   verbose=1, 
                   patience=10, 
                   min_delta=0.001, 
                   mode='max')
mc = ModelCheckpoint('auc_weight_NIH2_224.hdf5', 
                     monitor='val_auc', 
                     verbose=1, 
                     save_best_only=True, 
                     mode='max')
rlr = ReduceLROnPlateau(monitor='val_auc',
                        mode='max',
                        factor=0.1,
                        patience=2)

In [26]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0.9,nesterov=True), 
              metrics=['acc', 
                       tf.keras.metrics.AUC(name='auc'), 
                       tf.keras.metrics.Precision(name='precision'), 
                       tf.keras.metrics.Recall(name='recall')])

In [9]:
history = model.fit_generator(train_generator, 
                              epochs = 50, 
                              validation_data=validation_generator, 
                              class_weight = class_weights,
                             callbacks = [es, mc,rlr],
                             use_multiprocessing=False)

In [10]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [11]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [12]:
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model auc')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [13]:
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('insert_path')
plt.show()

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
test_generator = test_datagen.flow_from_directory('Test', 
                                                        target_size=(224, 224), 
                                                        class_mode= 'binary',
                                                        color_mode='rgb', 
                                                        batch_size=200,
                                                        interpolation='lanczos',
                                                       shuffle=False)
Y_pred = model.predict(test_generator, verbose = 1, steps = 10)
y_pred = np.round(Y_pred)
print('Confusion Matrix')
cm = confusion_matrix(test_generator.classes, y_pred)
target_names = ['COVID-Neg', 'COVID-Pos']
print(cm)
# disp = ConfusionMatrixDisplay(cm, display_labels=target_names)
# disp = disp.plot(cmap='Blues', values_format='.0f')
# plt.savefig('/deepcovidxr/Ramsey_Keras/Resnet_cm.pdf')
plt.show()
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [15]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(test_generator.classes, Y_pred)
auc = auc(fpr, tpr)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc))
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.show()

In [16]:
from tqdm import tqdm
#Confusion Matrix and Classification Report
test_generator = train_datagen.flow_from_directory('NMHFiles_224/Test', 
                                                        target_size=(224, 224), 
                                                        class_mode= 'binary',
                                                        color_mode='rgb', 
                                                        batch_size=200,
                                                        interpolation='lanczos',
                                                       shuffle=False)
tta_steps = 10
predictions = []
for i in tqdm(range(tta_steps)):
    preds = model.predict(test_generator, steps = 10, verbose=1)
    predictions.append(preds)
Y_pred = np.mean(predictions, axis=0)
# Y_pred = model.predict(test_generator, verbose = 1, steps = 10)
y_pred = np.round(Y_pred)
print('Confusion Matrix')
cm = confusion_matrix(test_generator.classes, y_pred)
print(cm)
target_names = ['COVID-Neg', 'COVID-Pos']
# disp = ConfusionMatrixDisplay(cm, display_labels=target_names)
# disp = disp.plot(cmap='Blues', values_format='.0f')
# plt.savefig('/deepcovidxr/Ramsey_Keras/Resnet_cm.pdf')
# plt.show()
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [17]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(test_generator.classes, Y_pred)
auc = auc(fpr, tpr)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc))
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.show()